## Import

In [1]:
from numpy import load
import numpy as np
import time 

## Load Data

In [2]:
ag2l = load('/Users/krol/research/Summer2022/Inversion Benchmark Files/ag1000g_2L_gambiae_full.npz')
counts = ag2l['allele_counts']
pos = ag2l['positions']
mask = ag2l['mask']

In [3]:
#create a csv of values so I can manually check that these values are correct that Im getting out of the array
for key, value in ag2l.items():
    np.savetxt("test" + key + ".csv", value, fmt='%f')

## Analyzing Data .. thinking of ideas

In [4]:
print('Counts', counts, counts.shape)
print("Pos", pos, pos.shape)
print("Mask", mask, mask.shape)

Counts [[0. 1. 1. ... 1. 1. 0.]
 [0. 2. 2. ... 0. 0. 1.]
 [0. 2. 2. ... 0. 0. 0.]
 ...
 [0. 2. 2. ... 0. 0. 0.]
 [0. 2. 2. ... 0. 0. 0.]
 [0. 2. 2. ... 1. 1. 0.]] (81, 596189)
Pos [   44889    50387   157120 ... 49356239 49356248 49356406] (596189,)
Mask [0. 0. 0. ... 0. 0. 0.] (596189,)


## Sliding Window

In [5]:
def slide_(allele_counts, windowsize, stepsize):
    #zz = np.zeros(windowsize[0], windowsize[1])
    alpos = []
    for x in range(0, allele_counts.shape[0], stepsize):
        for y in range(0, allele_counts.shape[1], stepsize):
            alcounts = allele_counts[x:x + windowsize[1], y:y+ windowsize[0]]
            if y % windowsize[0] == 0:
                alpos = (pos[y:y + windowsize[0]])
                almask = (mask[y:y + windowsize[0]])
            yield (alcounts, (alpos), (almask))

In [6]:
aa = slide_(counts, np.array([3,3]), 1)
bb = slide_(counts, np.array([3,3]), 1)
cc = slide_(counts, np.array([3,3]), 1)

In [7]:
alc_ = []
alp_ = []
alm_ = []

In [8]:
%%time
for _, _, almask in bb:
    #print("Print Orginal: ", almask)
    #print("Print modified: ", np.tile(almask, (1,3)))
    alm_.append(np.tile(almask, (1,3)))

#o = np.unique(temp, axis=0)
#for i in o:
#    alp_.append(np.tile([i], (3,1)))

CPU times: user 2min 3s, sys: 1.79 s, total: 2min 5s
Wall time: 2min 6s


In [9]:
%%time
counter = 0
for _, alpos, _ in aa:
    #print(alpos)
    alp_.append(np.tile(alpos, (3,1)))

CPU times: user 2min 2s, sys: 1.77 s, total: 2min 4s
Wall time: 2min 4s


In [10]:
%%time
for c,_,_ in cc:
    alc_.append(c)

CPU times: user 36 s, sys: 594 ms, total: 36.6 s
Wall time: 36.6 s


In [11]:
len(alp_)

48291309

In [12]:
len(alc_)

48291309

In [13]:
len(alm_)

48291309

In [ ]:
%%time
np.savez_compressed('test', a=alc_, b=alp_, c=alm_)